# HOMEWORK 5: TEXT CLASSIFICATION
In this homework, you will create models to classify texts from TRUE call-center. There are two classification tasks:
1. Action Classification: Identify which action the customer would like to take (e.g. enquire, report, cancle)
2. Object Classification: Identify which object the customer is referring to (e.g. payment, truemoney, internet, roaming) 

In this homework, you are asked to do the following tasks:
1. Data Cleaning
2. Preprocessing data for keras
3. Build and evaluate a model for "action" classification
4. Build and evaluate a model for "object" classification
5. Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go 


Note: we have removed phone numbers from the dataset for privacy purposes. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab\ Notebooks/NLP/hw6

/content/drive/MyDrive/Colab Notebooks/NLP/hw6


In [4]:
!wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv

--2021-02-27 17:58:25--  https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/37u83g55p19kvrl/clean-phone-data-for-students.csv [following]
--2021-02-27 17:58:26--  https://www.dropbox.com/s/raw/37u83g55p19kvrl/clean-phone-data-for-students.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca39390d253f174f0255cb1d3ec.dl.dropboxusercontent.com/cd/0/inline/BJveZ6jSV2Dup_MweZCMX61JF5zAqWF6YYwD3qQkud8tTUHn7Hfz72MvMgHLaHVmjScjcbzDyGtCplK2cvuyUvlYMVCz1dTp_KDK-ie8mzdYcw/file# [following]
--2021-02-27 17:58:26--  https://uca39390d253f174f0255cb1d3ec.dl.dropboxusercontent.com/cd/0/inline/BJveZ6jSV2Dup_MweZCMX61JF5zAqWF6YYwD3qQkud8tTUHn7Hfz72MvMgHLaHVmjScjc

## Import Libs

In [60]:
%matplotlib inline
import pandas
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from IPython.display import display

import matplotlib.pyplot as plt

## Loading data
First, we load the data from disk into a Dataframe.

A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.

In [36]:
data_df = pandas.read_csv('clean-phone-data-for-students.csv')

Let's preview the data.

In [37]:
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## Data cleaning

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes for object and 10 unique labels for action that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

Also note that, there are 13389 unqiue sentence utterances from 16175 utterances. You have to clean that too!

## #TODO 1: 
You will have to remove unwanted label duplications as well as duplications in text inputs. 
Also, you will have to trim out unwanted whitespaces from the text inputs. 
This shouldn't be too hard, as you have already seen it in the demo.



In [38]:
display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

In [39]:
# TODO1: Data cleaning
data_df.Action = data_df.Action.str.lower().copy()
data_df.Object = data_df.Object.str.lower().copy()

display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,8,26
top,บริการอื่นๆ,enquire,service
freq,97,10484,2528


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

array(['enquire', 'report', 'cancel', 'buy', 'activate', 'request',
       'garbage', 'change'], dtype=object)

In [40]:
data_df = data_df.drop_duplicates("Sentence Utterance", keep='first')
display(data_df.describe())

,Sentence Utterance,Action,Object
count,13389,13389,13389
unique,13389,8,26
top,สอบถามหน่อยอ่ะค่ะ ถ้าจะติดตั้งจานอ่ะค่ะ ต้องทำ...,enquire,service
freq,1,8658,2111


In [44]:
data_df = data_df.rename(columns={"Sentence Utterance": "input"})

In [45]:
action_labels = data_df.Action.unique()
object_labels = data_df.Object.unique()

action2index = dict(zip(action_labels, range(len(action_labels))))
index2action = dict(zip(range(len(action_labels)), action_labels))

object2index = dict(zip(object_labels, range(len(object_labels))))
index2object = dict(zip(range(len(object_labels)), object_labels))

display(action2index)
display(index2action)

display(object2index)
display(index2object)

{'activate': 4,
 'buy': 3,
 'cancel': 2,
 'change': 7,
 'enquire': 0,
 'garbage': 6,
 'report': 1,
 'request': 5}

{0: 'enquire',
 1: 'report',
 2: 'cancel',
 3: 'buy',
 4: 'activate',
 5: 'request',
 6: 'garbage',
 7: 'change'}

{'balance': 7,
 'balance_minutes': 18,
 'bill': 9,
 'contact': 24,
 'credit': 10,
 'detail': 8,
 'garbage': 20,
 'idd': 19,
 'information': 16,
 'internet': 3,
 'iservice': 13,
 'lost_stolen': 17,
 'loyalty_card': 23,
 'mobile_setting': 12,
 'nontruemove': 6,
 'officer': 25,
 'package': 1,
 'payment': 0,
 'phone_issues': 4,
 'promotion': 11,
 'rate': 22,
 'ringtone': 21,
 'roaming': 14,
 'service': 5,
 'suspend': 2,
 'truemoney': 15}

{0: 'payment',
 1: 'package',
 2: 'suspend',
 3: 'internet',
 4: 'phone_issues',
 5: 'service',
 6: 'nontruemove',
 7: 'balance',
 8: 'detail',
 9: 'bill',
 10: 'credit',
 11: 'promotion',
 12: 'mobile_setting',
 13: 'iservice',
 14: 'roaming',
 15: 'truemoney',
 16: 'information',
 17: 'lost_stolen',
 18: 'balance_minutes',
 19: 'idd',
 20: 'garbage',
 21: 'ringtone',
 22: 'rate',
 23: 'loyalty_card',
 24: 'contact',
 25: 'officer'}

## #TODO 2: Preprocessing data for Keras
You will be using Tensorflow 2 keras in this assignment. Please show us how you prepare your data for keras.
Don't forget to split data into train and test sets (+ validation set if you want)

In [46]:
# TODO2: Preprocessing data for Keras
data = data_df.to_numpy()

array(['enquire', 'enquire', 'report', ..., 'enquire', 'enquire',
       'enquire'], dtype=object)

In [48]:
data[:,1] = np.vectorize(action2index.get)(data[:,1])
data[:,2] = np.vectorize(object2index.get)(data[:,2])

In [49]:
data

array([[' <PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counter Services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท',
        0, 0],
       [' internet ยังความเร็วอยุ่เท่าไหร ครับ', 0, 1],
       [' ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ', 1, 2],
       ...,
       ['ยอดเงินเหลือเท่าไหร่ค่ะ', 0, 7],
       ['ยอดเงินในระบบ', 0, 7],
       ['สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ', 0, 1]], dtype=object)

In [56]:
X = data[:,0].copy()
Y = data[:,1:3].copy()

print(X.shape)
print(Y.shape)

(13389,)
(13389, 2)


In [62]:
x_train, x_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.2)

ya_train, ya_test = Y_train[:,0].copy(), Y_test[:,0].copy()
yo_train, yo_test = Y_train[:,1].copy(), Y_test[:,1].copy()

print(ya_train.shape, yo_train.shape)
print(np.max(ya_train))
print(np.max(yo_train))

(10711,) (10711,)
7
25


## #TODO 3: Build and evaluate a model for "action" classification


In [ ]:
#TODO 3: Build and evaluate a model for "action" classification

## #TODO 4: Build and evaluate a model for "object" classification



In [ ]:
#TODO 4: Build and evaluate a model for "object" classification

## #TODO 5: Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go 

This can be a bit tricky, if you are not familiar with the Keras functional API. PLEASE READ these webpages(https://www.tensorflow.org/guide/keras/functional, https://keras.io/getting-started/functional-api-guide/) before you start this task.   

Your model will have 2 separate output layers one for action classification task and another for object classification task. 

This is a rough sketch of what your model might look like:
![image](https://raw.githubusercontent.com/ekapolc/nlp_course/master/HW5/multitask_sketch.png)

In [ ]:
#TODO 5: Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go